In [4]:
import torch
import json
from pathlib import Path
from task.taskManager import get_model

def load_checkpoint(checkpoint_path):
    checkpoint_path = Path(checkpoint_path)
    assert checkpoint_path.exists()
    checkpoint = torch.load(checkpoint_path)
    task = get_model(checkpoint["arguments"])
    return checkpoint["model"], checkpoint["history"], task


In [6]:
model, history, task = load_checkpoint("saves/warpTask-0-batch-1000.pth")

In [ ]:
for batch_num, x, y in model.dataloader:
    